In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from nltk.translate.bleu_score import sentence_bleu
from numpy import array
import pickle
import numpy as np
import os
import cv2
import json

ModuleNotFoundError: No module named 'pandas'

In [2]:
!pip install pandas

^C


In [ ]:
!pip install pandas nltk tqdm opencv-python

In [11]:
tf.test.is_gpu_available()

True

In [ ]:
!apt-get update && apt-get upgrade
!apt-get install -y python3-opencv
!apt-get install libgl1 -y

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1575 B]
Err:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Reading package lists... Done                                       
W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRele

Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libxau6 amd64 1:1.0.8-1ubuntu1 [7556 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libbsd0 amd64 0.8.7-1ubuntu0.1 [41.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxdmcp6 amd64 1:1.1.2-3 [10.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libxcb1 amd64 1.13-2~ubuntu18.04 [45.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libx11-data all 2:1.6.4-3ubuntu0.4 [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libx11-6 amd64 2:1.6.4-3ubuntu0.4 [572 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxext6 amd64 2:1.3.3-1 [29.4 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic/main amd64 ucf all 3.0038 [50.5 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/main amd64 fonts-dejavu-core all 2.37-1 [1041 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic/main amd64 fontconfig-config all 2.12.6-0ubuntu2 [55.8

Get:80 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libarmadillo8 amd64 1:8.400.0+dfsg-2 [81.9 kB]
Get:81 http://archive.ubuntu.com/ubuntu bionic/main amd64 libatk1.0-data all 2.28.1-1 [2992 B]
Get:82 http://archive.ubuntu.com/ubuntu bionic/main amd64 libatk1.0-0 amd64 2.28.1-1 [43.9 kB]
Get:83 http://archive.ubuntu.com/ubuntu bionic/main amd64 libatk-bridge2.0-0 amd64 2.26.2-1 [57.3 kB]
Get:84 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libavahi-common-data amd64 0.7-3.1ubuntu1.3 [22.2 kB]
Get:85 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libavahi-common3 amd64 0.7-3.1ubuntu1.3 [21.6 kB]
Get:86 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libavahi-client3 amd64 0.7-3.1ubuntu1.3 [25.2 kB]
Get:87 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libva2 amd64 2.1.0-3 [47.6 kB]
Get:88 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libva-drm2 amd64 2.1.0-3 [6880 B]
Get:89 http://archive.ubuntu.com/ubuntu bionic/main amd64 

Get:160 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libkmlengine1 amd64 1.3.0-5 [73.7 kB]
Get:161 http://archive.ubuntu.com/ubuntu bionic/main amd64 mysql-common all 5.8+1.0.4 [7308 B]
Get:162 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmysqlclient20 amd64 5.7.38-0ubuntu0.18.04.1 [689 kB]
Get:163 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libnetcdf13 amd64 1:4.6.0-2build1 [333 kB]
Get:164 http://archive.ubuntu.com/ubuntu bionic/main amd64 libltdl7 amd64 2.4.6-2 [38.8 kB]
Get:165 http://archive.ubuntu.com/ubuntu bionic/main amd64 libodbc1 amd64 2.3.4-1.1ubuntu3 [183 kB]
Get:166 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libogdi3.2 amd64 3.2.0+ds-2 [213 kB]
Get:167 http://archive.ubuntu.com/ubuntu bionic/main amd64 libnspr4 amd64 2:4.18-1ubuntu1 [112 kB]
Get:168 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libnss3 amd64 2:3.35-2ubuntu2.13 [1220 kB]
Get:169 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpopp

Get:235 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libopencv-photo3.2 amd64 3.2.0+dfsg-4ubuntu0.1 [188 kB]
Get:236 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libopencv-video3.2 amd64 3.2.0+dfsg-4ubuntu0.1 [122 kB]
Get:237 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libopencv-shape3.2 amd64 3.2.0+dfsg-4ubuntu0.1 [56.3 kB]
Get:238 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libopencv-stitching3.2 amd64 3.2.0+dfsg-4ubuntu0.1 [166 kB]
Get:239 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libopencv-superres3.2 amd64 3.2.0+dfsg-4ubuntu0.1 [42.4 kB]
Get:240 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libopencv-videostab3.2 amd64 3.2.0+dfsg-4ubuntu0.1 [97.7 kB]
Get:241 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libhwloc5 amd64 1.11.9-1 [105 kB]
Get:242 http://archive.ubuntu.com/ubuntu bionic/main amd64 ocl-icd-libopencl1 amd64 2.2.11-1ubuntu1 [30.3 kB]
Get:243 http://ar

Selecting previously unselected package shared-mime-info.
Preparing to unpack .../026-shared-mime-info_1.9-2_amd64.deb ...
Unpacking shared-mime-info (1.9-2) ...
Selecting previously unselected package tzdata.
Preparing to unpack .../027-tzdata_2022a-0ubuntu0.18.04_all.deb ...
Unpacking tzdata (2022a-0ubuntu0.18.04) ...
Selecting previously unselected package xkb-data.
Preparing to unpack .../028-xkb-data_2.23.1-1ubuntu1.18.04.1_all.deb ...
Unpacking xkb-data (2.23.1-1ubuntu1.18.04.1) ...
Selecting previously unselected package libdrm-common.
Preparing to unpack .../029-libdrm-common_2.4.101-2~18.04.1_all.deb ...
Unpacking libdrm-common (2.4.101-2~18.04.1) ...
Selecting previously unselected package libdrm2:amd64.
Preparing to unpack .../030-libdrm2_2.4.101-2~18.04.1_amd64.deb ...
Unpacking libdrm2:amd64 (2.4.101-2~18.04.1) ...
Selecting previously unselected package libedit2:amd64.
Preparing to unpack .../031-libedit2_3.1-20170329-1_amd64.deb ...
Unpacking libedit2:amd64 (3.1-20170329

Selecting previously unselected package libnl-route-3-200:amd64.
Preparing to unpack .../071-libnl-route-3-200_3.2.29-0ubuntu3_amd64.deb ...
Unpacking libnl-route-3-200:amd64 (3.2.29-0ubuntu3) ...
Selecting previously unselected package libibverbs1:amd64.
Preparing to unpack .../072-libibverbs1_17.1-1ubuntu0.2_amd64.deb ...
Unpacking libibverbs1:amd64 (17.1-1ubuntu0.2) ...
Selecting previously unselected package ibverbs-providers:amd64.
Preparing to unpack .../073-ibverbs-providers_17.1-1ubuntu0.2_amd64.deb ...
Unpacking ibverbs-providers:amd64 (17.1-1ubuntu0.2) ...
Selecting previously unselected package libaacs0:amd64.
Preparing to unpack .../074-libaacs0_0.9.0-1_amd64.deb ...
Unpacking libaacs0:amd64 (0.9.0-1) ...
Selecting previously unselected package libblas3:amd64.
Preparing to unpack .../075-libblas3_3.7.1-4ubuntu1_amd64.deb ...
Unpacking libblas3:amd64 (3.7.1-4ubuntu1) ...
Selecting previously unselected package liblapack3:amd64.
Preparing to unpack .../076-liblapack3_3.7.1-4u

Unpacking libchromaprint1:amd64 (1.4.3-1) ...
Selecting previously unselected package libgme0:amd64.
Preparing to unpack .../118-libgme0_0.6.2-1_amd64.deb ...
Unpacking libgme0:amd64 (0.6.2-1) ...
Selecting previously unselected package libmpg123-0:amd64.
Preparing to unpack .../119-libmpg123-0_1.25.10-1_amd64.deb ...
Unpacking libmpg123-0:amd64 (1.25.10-1) ...
Selecting previously unselected package libvorbisfile3:amd64.
Preparing to unpack .../120-libvorbisfile3_1.3.5-4.2_amd64.deb ...
Unpacking libvorbisfile3:amd64 (1.3.5-4.2) ...
Selecting previously unselected package libopenmpt0:amd64.
Preparing to unpack .../121-libopenmpt0_0.3.6-1_amd64.deb ...
Unpacking libopenmpt0:amd64 (0.3.6-1) ...
Selecting previously unselected package libssh-gcrypt-4:amd64.
Preparing to unpack .../122-libssh-gcrypt-4_0.8.0~20170825.94fa1e38-1ubuntu0.7_amd64.deb ...
Unpacking libssh-gcrypt-4:amd64 (0.8.0~20170825.94fa1e38-1ubuntu0.7) ...
Selecting previously unselected package libavformat57:amd64.
Prepari

Selecting previously unselected package libogdi3.2.
Preparing to unpack .../165-libogdi3.2_3.2.0+ds-2_amd64.deb ...
Unpacking libogdi3.2 (3.2.0+ds-2) ...
Selecting previously unselected package libnspr4:amd64.
Preparing to unpack .../166-libnspr4_2%3a4.18-1ubuntu1_amd64.deb ...
Unpacking libnspr4:amd64 (2:4.18-1ubuntu1) ...
Selecting previously unselected package libnss3:amd64.
Preparing to unpack .../167-libnss3_2%3a3.35-2ubuntu2.13_amd64.deb ...
Unpacking libnss3:amd64 (2:3.35-2ubuntu2.13) ...
Selecting previously unselected package libpoppler73:amd64.
Preparing to unpack .../168-libpoppler73_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking libpoppler73:amd64 (0.62.0-2ubuntu2.12) ...
Selecting previously unselected package libpq5:amd64.
Preparing to unpack .../169-libpq5_10.19-0ubuntu0.18.04.1_amd64.deb ...
Unpacking libpq5:amd64 (10.19-0ubuntu0.18.04.1) ...
Selecting previously unselected package libqhull7:amd64.
Preparing to unpack .../170-libqhull7_2015.2-4_amd64.deb ...
Unpacking libqh

In [3]:
%%capture captured
model=ResNet50(include_top=False, weights='imagenet',pooling='avg',input_shape=(224,224,3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-05-08 18:17:32.555859: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-08 18:17:36.708506: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-08 18:17:36.708571: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kabilan-pt4987): /proc/driver/nvidia/version does not exist
2022-05-08 18:17:36.726601: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-08 18:17:36.759983: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2599990000 Hz
2022-05-08 18:17:36.761901: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56253b170810 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-08

In [4]:
train_path='/home/kabilan/Desktop/caption/sample/image_caption_test_dict.json'

In [15]:
train_path='../sample/image_caption_test_dict.json'

In [5]:
train_captions = json.load(open(train_path,'r'))

In [6]:
path = "/home/kabilan/Desktop/caption/dataset/images/"

In [7]:
for each in train_captions.keys():
    captions = train_captions[each]
    for i in range(len(captions)):
        captions[i] =  "<start> " + f"{captions[i]}".strip() + " <end>"

In [8]:
train_features={}
c=0
for image_name in tqdm(train_captions):
  img_path=path+image_name
  img=image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  train_features[image_name]=features.squeeze()

100%|████████████████████████████████████████████████████████████████| 1000/1000 [01:00<00:00, 16.43it/s]


In [ ]:
train_features

In [9]:
all_captions=[]
for img_id in train_captions:
  for captions in train_captions[img_id]:
    all_captions.append(captions)
all_words=" ".join(all_captions)
print(len(all_words))
unique_words=list(set(all_words.strip().split(" ")))
print(len(unique_words))

332863
2648


In [10]:
vocab_size=len(unique_words)+1
max_length=40

In [11]:
words_to_indices={val:index+1 for index, val in enumerate(unique_words)}
indices_to_words = { index+1:val for index, val in enumerate(unique_words)}
words_to_indices["Unk"]=0
indices_to_words[0]="Unk"

In [12]:
train_encoded_captions={}
for img_id in tqdm(train_captions):
  train_encoded_captions[img_id]=[]
  for i in range(5):
    train_encoded_captions[img_id].append([words_to_indices[s] for s in train_captions[img_id][i].split(" ")])

100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 46323.38it/s]


In [13]:
for img_id in tqdm(train_encoded_captions):
  train_encoded_captions[img_id]=pad_sequences(train_encoded_captions[img_id], maxlen=max_length, padding='post')

100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 18717.97it/s]


In [14]:
def data_generator(train_encoded_captions,train_features,num_of_photos):
  X1, X2, Y = list(), list(), list()
  max_length=40
  n=0
  for img_id in train_encoded_captions:
    n+=1
    for i in range(5):
      for j in range(1,40):
        curr_sequence=train_encoded_captions[img_id][i][0:j].tolist()
        next_word=train_encoded_captions[img_id][i][j]
        curr_sequence=pad_sequences([curr_sequence], maxlen=max_length, padding='post')[0]
        one_hot_next_word=to_categorical([next_word],vocab_size)[0]
        X1.append(train_features[img_id])
        X2.append(curr_sequence)
        Y.append(one_hot_next_word)
    if(n==num_of_photos):
      yield [[array(X1), array(X2)], array(Y)]
      X1, X2, Y = list(), list(), list()
      n=0

In [15]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.recurrent import  LSTM
from keras.layers.merge import add
from keras.layers.embeddings import Embedding

In [ ]:
input_1=Input(shape=(2048,))
dropout_1=Dropout(0.2)(input_1)
dense_1=Dense(256,activation='relu')(dropout_1)

input_2=Input(shape=(max_length,))
embedding_1=Embedding(vocab_size,256)(input_2)
dropout_2=Dropout(0.2)(embedding_1)
lstm_1=LSTM(256)(dropout_2)

add_1=add([dense_1,lstm_1])
dense_2=Dense(256,activation='relu')(add_1)
dense_3=Dense(vocab_size,activation='softmax')(dense_2)

model=Model(inputs=[input_1,input_2],outputs=dense_3)

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
epochs=10
no_of_photos=1
steps=len(train_encoded_captions)
for i in range(epochs):
  generator=data_generator(train_encoded_captions,train_features,no_of_photos)
  model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)

In [ ]:
def greedy_search(photo):
  photo=photo.reshape(1,2048)
  in_text='<start>'
  for i in range(max_length):
    sequence = [words_to_indices[s] for s in in_text.split(" ") if s in words_to_indices]
    sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
    y_pred = model.predict([photo,sequence],verbose=0)
    y_pred = np.argmax(y_pred[0])
    word = indices_to_words[y_pred]
    in_text += ' ' + word
    if word == '<end>':
      break
  final = in_text.split()
  final = final[1:-1]
  return final

In [ ]:
for img_id in list(train_features.keys())[0:20]:
    img=cv2.imread(path+img_id)
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB));
    photo=train_features[img_id]
    candidate=greedy_search(photo)
    print(candidate)
    plt.show()